In [1]:
import pandas as pd

Problem Statement: https://www.kaggle.com/c/house-prices-advanced-regression-techniques
Dataset: https://github.com/abulbasar/data/tree/master/kaggle-houseprice

In [2]:
df = pd.read_csv("/data/kaggle/data_combined_cleaned.csv")
df = df.drop(columns=["Id"])
df = df[~df.SalesPrice.isnull()]
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalesPrice
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,Inside,Gtl,...,0,None,None,None,0,2,2008,WD,Normal,208500.0
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,FR2,Gtl,...,0,None,None,None,0,5,2007,WD,Normal,181500.0
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,Inside,Gtl,...,0,None,None,None,0,9,2008,WD,Normal,223500.0
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,Corner,Gtl,...,0,None,None,None,0,2,2006,WD,Abnorml,140000.0
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,FR2,Gtl,...,0,None,None,None,0,12,2008,WD,Normal,250000.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Data columns (total 79 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
MasVnrType       1460 no

# Exercise:
- Build a model to predict the SalesPrice using other columns as features. Split the dataset (1460 records) into training and test sets with 70/30 ratio. Take log of SalesPrice for prediction to reduce the impact of the outliers.
- What is the R2 score and rmse score? 
- Fit Lasso model. Test a few alpha values (>0) and see which gives a good r2 score. 

In [4]:
import numpy as np
from sklearn import preprocessing, model_selection, linear_model, metrics, pipeline

In [5]:
target = "SalesPrice"
X = df.drop(columns=[target])
X = pd.get_dummies(X, drop_first=True)
columns = X.columns
X = X.values.astype("float")
y = np.log(df[target])

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y,
                           test_size = 0.3, random_state = 1)

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1
                                    , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.LinearRegression())
])


pipe.fit(X_train, y_train)

y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print("training r2", metrics.r2_score(y_train, y_train_pred))
print("test r2", metrics.r2_score(y_test, y_test_pred))

print("training rmse:"
    , np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("test rmse:"
      , np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))
est = pipe.steps[-1][-1]
pd.DataFrame({"feature": columns, "coefficient": est.coef_})

training r2 0.9494518297088844
test r2 -3.0715943247509893e+22
training rmse: 0.08606817747710854
test rmse: 76190488351.17424


,feature,coefficient
0,MSSubClass,1.248159e-02
1,LotFrontage,1.963124e-03
2,LotArea,3.151273e-02
3,OverallQual,5.675519e-02
4,OverallCond,3.468973e-02
5,YearBuilt,5.024337e-02
6,YearRemodAdd,1.540301e-02
7,MasVnrArea,-3.995381e-03
8,BsmtFinSF1,4.530257e+10
9,BsmtFinSF2,1.612199e+10


In [6]:
target = "SalesPrice"
X = df.drop(columns=[target])
X = pd.get_dummies(X, drop_first=True)
columns = X.columns
X = X.values.astype("float")
y = np.log(df[target])

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y,
                           test_size = 0.3, random_state = 1)

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1
                                    , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Lasso(alpha=0.01, random_state=1))
])


pipe.fit(X_train, y_train)

y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print("training r2", metrics.r2_score(y_train, y_train_pred))
print("test r2", metrics.r2_score(y_test, y_test_pred))

print("training rmse:"
    , np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("test rmse:"
      , np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))
est = pipe.steps[-1][-1]


summary = pd.DataFrame({"feature": columns, "coefficient": est.coef_})
summary["coef_mag"] = np.abs(summary.coefficient)
summary = summary.sort_values("coef_mag", ascending = False)
summary = summary[summary.coef_mag>0]
summary

training r2 0.8940480673803641
test r2 0.8785389987905549
training rmse: 0.12460754170603186
test rmse: 0.15150859029885974


,feature,coefficient,coef_mag
3,OverallQual,0.104135,0.104135
15,GrLivArea,0.095563,0.095563
25,GarageCars,0.055896,0.055896
5,YearBuilt,0.029465,0.029465
39,MSZoning_RM,-0.026432,0.026432
4,OverallCond,0.023309,0.023309
6,YearRemodAdd,0.021713,0.021713
70,Neighborhood_NridgHt,0.020148,0.020148
60,Neighborhood_Crawfor,0.019489,0.019489
16,BsmtFullBath,0.018030,0.018030


In [7]:
summary = pd.DataFrame({"feature": columns, "coefficient": est.coef_})
summary["coef_mag"] = np.abs(summary.coefficient)
summary = summary.sort_values("coef_mag", ascending = False)
summary = summary[summary.coef_mag>0]
summary

,feature,coefficient,coef_mag
3,OverallQual,0.104135,0.104135
15,GrLivArea,0.095563,0.095563
25,GarageCars,0.055896,0.055896
5,YearBuilt,0.029465,0.029465
39,MSZoning_RM,-0.026432,0.026432
4,OverallCond,0.023309,0.023309
6,YearRemodAdd,0.021713,0.021713
70,Neighborhood_NridgHt,0.020148,0.020148
60,Neighborhood_Crawfor,0.019489,0.019489
16,BsmtFullBath,0.018030,0.018030


In [22]:
target = "SalesPrice"
X = df.drop(columns=[target])
X = pd.get_dummies(X, drop_first=True)
columns = X.columns
X = X.values.astype("float")
y = np.log(df[target])

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y,
                           test_size = 0.3, random_state = 147595)

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1
                                    , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Ridge(alpha=40, random_state=1))
])


pipe.fit(X_train, y_train)

y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print("training r2", metrics.r2_score(y_train, y_train_pred))
print("test r2", metrics.r2_score(y_test, y_test_pred))

print("training rmse:"
    , np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("test rmse:"
      , np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))
est = pipe.steps[-1][-1]


summary = pd.DataFrame({"feature": columns, "coefficient": est.coef_})
summary["coef_mag"] = np.abs(summary.coefficient)
summary = summary.sort_values("coef_mag", ascending = False)
summary = summary[summary.coef_mag>0]
summary

training r2 0.9468756989746974
test r2 0.6160944791413361
training rmse: 0.09374379292490413
test rmse: 0.23569702086858121


,feature,coefficient,coef_mag
235,PoolQC_Gd,-0.074129,0.074129
15,GrLivArea,0.062381,0.062381
3,OverallQual,0.053154,0.053154
38,MSZoning_RL,0.050967,0.050967
90,Condition2_PosN,-0.046524,0.046524
12,1stFlrSF,0.045143,0.045143
4,OverallCond,0.039686,0.039686
39,MSZoning_RM,0.039352,0.039352
11,TotalBsmtSF,0.036887,0.036887
13,2ndFlrSF,0.036532,0.036532


In [9]:
from sklearn import feature_selection

In [18]:
scaler = preprocessing.StandardScaler()
X_train_std = scaler.fit_transform(X_train)
_, pvals = feature_selection.f_regression(X_train_std, y_train)
summary = pd.DataFrame({"feature": columns, "pval": pvals})
summary = summary.sort_values("pval")
summary = summary[summary.pval < 0.01] 
# pval < 0.01 are considered significant
# closer to 0 is more significant features
summary.iloc[:10,:]

/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,feature,pval
3,OverallQual,1.543610e-232
15,GrLivArea,1.502663e-147
25,GarageCars,8.980028e-140
26,GarageArea,9.673012e-121
18,FullBath,7.435435e-106
11,TotalBsmtSF,4.707153e-98
151,ExterQual_TA,1.996594e-96
12,1stFlrSF,1.957045e-95
5,YearBuilt,2.946855e-85
6,YearRemodAdd,3.840149e-80


In [21]:
target = "SalesPrice"
X = df.drop(columns=[target])
X = pd.get_dummies(X, drop_first=True)
columns = X.columns
X = X.values.astype("float")
y = np.log(df[target])

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y,
                           test_size = 0.3, random_state = 1)

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1
                                    , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Lasso(alpha=0.07, random_state=1))
])


pipe.fit(X_train, y_train)

y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print("training r2", metrics.r2_score(y_train, y_train_pred))
print("test r2", metrics.r2_score(y_test, y_test_pred))

print("training rmse:"
    , np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("test rmse:"
      , np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))
est = pipe.steps[-1][-1]


summary = pd.DataFrame({"feature": columns, "coefficient": est.coef_})
summary["coef_mag"] = np.abs(summary.coefficient)
summary = summary.sort_values("coef_mag", ascending = False)
summary = summary[summary.coef_mag>0]
summary

training r2 0.7395929485859631
test r2 0.746971064732701
training rmse: 0.19535127802592958
test rmse: 0.2186774107558645


,feature,coefficient,coef_mag
3,OverallQual,0.140629,0.140629
15,GrLivArea,0.069171,0.069171
25,GarageCars,0.055262,0.055262
11,TotalBsmtSF,0.014343,0.014343
5,YearBuilt,0.012730,0.012730
6,YearRemodAdd,0.009382,0.009382
12,1stFlrSF,0.006673,0.006673
39,MSZoning_RM,-0.005534,0.005534
23,Fireplaces,0.000774,0.000774


In [25]:
pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1
                                    , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Ridge(alpha=40, random_state=1))
])

scores = model_selection.cross_val_score(pipe, X=X_train, y=y_train, cv=5)
scores

array([0.89012794, 0.70703003, 0.81430385, 0.83800989, 0.87462756])

In [26]:
np.mean(scores)

0.8248198555818318

In [27]:
pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1
                                    , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Ridge(alpha=40, random_state=1))
])

scores = model_selection.cross_val_score(pipe, X=X_train, y=y_train, cv=5)
np.mean(scores)

0.8248198555818318

In [28]:
pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1
                                    , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Lasso(alpha=0.07, random_state=1))
])

scores = model_selection.cross_val_score(pipe, X=X_train, y=y_train, cv=5)
np.mean(scores)

0.746624200157168

In [29]:
# Hyper parameter tuning

In [41]:
np.linspace(0, 1, 10)

array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ])

In [42]:
np.linspace(0, 0.1, 10)

array([0.        , 0.01111111, 0.02222222, 0.03333333, 0.04444444,
       0.05555556, 0.06666667, 0.07777778, 0.08888889, 0.1       ])

In [39]:
target = "SalesPrice"
X = df.drop(columns=[target])
X = pd.get_dummies(X, drop_first=True)
columns = X.columns
X = X.values.astype("float")
y = np.log(df[target])

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y,
                           test_size = 0.3, random_state = 1)

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1
                                    , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.ElasticNet(alpha=0.07, max_iter=1000
                                    , random_state=1))
])

param_grid = {
    "est__alpha": np.linspace(0, 0.1, 10),
    "est__l1_ratio": np.linspace(0, 1, 10)
}

gsearch = model_selection.GridSearchCV(pipe, param_grid,
                                cv =5, verbose = True)
gsearch.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:267: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:477: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:267: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimato

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:267: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:477: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:267: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:267: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:477: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:267: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:267: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:477: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/l

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=1, include_bias=False, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', ElasticNet(alpha=0.07, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=1, selection='cyclic', tol=0.0001, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'est__alpha': array([0.     , 0.01111, 0.02222, 0.03333, 0.04444, 0.05556, 0.06667,
       0.07778, 0.08889, 0.1    ]), 'est__l1_ratio': array([0.     , 0.11111, 0.22222, 0.33333, 0.44444, 0.55556, 0.66667,
       0.77778, 0.88889, 1.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [40]:
gsearch.best_params_

{'est__alpha': 0.07777777777777778, 'est__l1_ratio': 0.1111111111111111}

In [43]:
gsearch.best_estimator_

Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=1, include_bias=False, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', ElasticNet(alpha=0.07777777777777778, copy_X=True, fit_intercept=True,
      l1_ratio=0.1111111111111111, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=1, selection='cyclic',
      tol=0.0001, warm_start=False))])

In [45]:
y_train_pred = gsearch.predict(X_train)
y_test_pred = gsearch.predict(X_test)

print("training r2", metrics.r2_score(y_train, y_train_pred))
print("test r2", metrics.r2_score(y_test, y_test_pred))

print("training rmse:"
    , np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("test rmse:"
      , np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

training r2 0.8973696343723988
test r2 0.8768660857303611
training rmse: 0.12263878092804865
test rmse: 0.1525484051834084


In [46]:
import pickle

In [47]:
with open("model.pkl", "wb") as f:
    pickle.dump(gsearch, f)

In [48]:
with open("model.pkl", "rb") as f:
    model = pickle.load(f)

In [50]:
%%time 

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print("training r2", metrics.r2_score(y_train, y_train_pred))
print("test r2", metrics.r2_score(y_test, y_test_pred))

print("training rmse:"
    , np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("test rmse:"
      , np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

training r2 0.8973696343723988
test r2 0.8768660857303611
training rmse: 0.12263878092804865
test rmse: 0.1525484051834084
CPU times: user 46.1 ms, sys: 2.02 ms, total: 48.1 ms
Wall time: 12.4 ms
